# Image registration for Nature reserve

As it is a sequoia orthomosaic, scaled from 0-to-1, we assume it is calibrated
* For the following Dataset: https://zenodo.org/records/3063335
* Using code snippets from:

Output: `uint16` 0-65535 scaled .tif files with green, red, rededge, nir bands


In [1]:
import os
import rioxarray as rxr
import xarray as xr
import tifffile
from glob import glob
import numpy as np
from itertools import repeat

In [5]:
def load_image(path):
    return rxr.open_rasterio(path), path
    
def rescale_to_uint16(da):
    da = da.where(~np.isnan(da), 65535)
    rescaled = (da * 65535).clip(0, 65535).astype(np.uint16)
    
    return rescaled

def rescale_float32_to_uint16_xarray(xarr):
    # Mask NaNs and -1000 values
    xarr = xr.where(xarr == -99999, 1, xarr)
    
    # Find min and max values ignoring NaNs
    min_val = 0
    max_val = 1
    
    # Normalize to [0, 1] ignoring NaNs
    normalized = (xarr - min_val) / (max_val - min_val)
    
    # Scale to [0, 65534]
    scaled = (normalized * 65535).astype(np.uint16)
    
    # Set NaNs and -1000 to 65535
    scaled = scaled.where(~xarr.isnull() , 65535)
    
    return scaled
    
def create_4_channel_image(green, red,rededge, nir):
    """Create a 4-channel image (G, R, RE, NIR)."""
    return np.stack((green, red, rededge, nir), axis=-1)
    

def save_image(filename, image, metadata=None):
    height, width, channels = image.shape
    with tiff.memmap(filename, shape=(height, width, channels), dtype=np.uint16) as memmap_tif:
        for start_row in range(0, height, chunk_size):
            end_row = min(start_row + chunk_size, height)
            memmap_tif[start_row:end_row, :, :] = image[start_row:end_row, :, :]


def process_multispec_set(green_image_path, output_directory):

    base_name = os.path.basename(green_image_path).replace("green.data.tif", "")
    # Load the green band image (reference image)
    dirname = os.path.dirname(green_image_path)
    
    tif,_ = load_image(green_image_path)
    green = tif.sel(band=1)
    red = tif.sel(band=2)
    rededge = tif.sel(band=3)
    nir = tif.sel(band=4)
    green = rescale_float32_to_uint16_xarray(green)
    red = rescale_float32_to_uint16_xarray(red)
    rededge = rescale_float32_to_uint16_xarray(rededge)
    nir = rescale_float32_to_uint16_xarray(nir)

    
     # Create a 4-channel image
    image_4ch = create_4_channel_image(green, red, rededge, nir)

    sensor = "SEQUOIA"
    cal = "CAL"
    set_title = os.path.basename(os.path.normpath(output_directory))
    output_filename = f"{sensor}_{cal}_{set_title}_{base_name}.tif"
    output_path = os.path.join(output_directory, output_filename)
    os.makedirs(output_directory, exist_ok=True)


    # Save the 4-channel image
    save_image(output_path, image_4ch)

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def process_directory(input_dir, output_directory, n_threads= 1):
    """Process an entire directory with multiple sets of multispectral images."""
    input_dir = os.path.abspath(input_dir)
    os.makedirs(output_directory, exist_ok=True)

    # Find all green band images (reference images)
    green_images = glob(os.path.join(input_dir, "*.tif"))

    # Process sets of 4 multispectral images in parallel with a progress bar
    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        list(tqdm(executor.map(process_multispec_set, green_images, repeat(output_directory)), total=len(green_images)))

    print("All images successfully processed.")

In [ ]:
folder = "../data/Sequoia_nature_reserve"
input_folder = folder
output_folder = "../data/processed/sequioa_nature_reserve/"
process_directory(input_folder, output_folder)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

In [31]:
tif,_ = load_image(glob(os.path.join(input_folder, "*.tif"))[0])
green = tif.sel(band=1)
red = tif.sel(band=2)
rededge = tif.sel(band=3)
nir = tif.sel(band=4)

In [33]:
green = rescale_float32_to_uint16_xarray(green)
red = rescale_float32_to_uint16_xarray(red)
rededge = rescale_float32_to_uint16_xarray(rededge)
nir = rescale_float32_to_uint16_xarray(nir)

In [38]:
green.values.max()

np.uint16(65535)

In [39]:
red.values.max()

np.uint16(65535)

In [48]:
im = create_4_channel_image(green, red, rededge, nir)

In [60]:
filename = "output.tif"

with tifffile.TiffWriter(filename) as tiff:
    tiff.write(im.astype(np.uint32), photometric='rgb')


error: 'I' format requires 0 <= number <= 4294967295